# Scratchpad for paper revisions

In [93]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path

from utils.data import make_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_model
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
dict(sorted(vars(args).items(),key=lambda k: k[0]))


{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 2,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'device': device(type='cpu'),
 'gating': 'oja_ctx',
 'hebb_normaliser': 10.0,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.01,
 'lrate_sgd': 0.01,
 'n_episodes': 200,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

## hyperparameter optimisation
I've changed the oja update to be applied to all hidden units. Now networks no longer converge.
Let's run HPO to find best-fitting lrate etc

In [96]:
args = parser.parse_args(args=[])
args.gating = "oja"
dict(sorted(vars(args).items(),key=lambda k: k[0]))

{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 2,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'gating': 'oja',
 'hebb_normaliser': 10.0,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.01,
 'lrate_sgd': 0.01,
 'n_episodes': 200,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

In [ ]:
# init tuner
tuner = HPOTuner(args, time_budget=60*30, metric="acc")

In [ ]:
# HPO on blocked with oja (all units)
tuner.tune(n_samples=500)

In [103]:
tuner.results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 29643_00000 to 29643_00499
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mean_loss                 500 non-null    float64
 1   mean_acc                  500 non-null    float64
 2   time_this_iter_s          500 non-null    float64
 3   done                      500 non-null    bool   
 4   timesteps_total           0 non-null      object 
 5   episodes_total            0 non-null      object 
 6   training_iteration        500 non-null    int64  
 7   neg_mean_loss             500 non-null    float64
 8   experiment_id             500 non-null    object 
 9   date                      500 non-null    object 
 10  timestamp                 500 non-null    int64  
 11  time_total_s              500 non-null    float64
 12  pid                       500 non-null    int64  
 13  hostname                  500 non-null    object 
 1

In [121]:
# tuner.results.sort_values("mean_acc",ascending=False).head(20)
df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","done"]]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
df.head(15)

,mean_loss,mean_acc,config.lrate_sgd,config.lrate_hebb,config.ctx_scaling
trial_id,,,,,
29643_00246,-29.047512,1.000,0.059468,0.000203,7
29643_00158,-29.011238,0.975,0.044441,0.000212,6
29643_00010,-28.901655,1.000,0.066424,0.000270,6
29643_00397,-28.890070,0.975,0.020580,0.000212,2
29643_00211,-28.881416,0.975,0.031433,0.000173,6
29643_00361,-28.840702,0.975,0.071419,0.000204,4
29643_00387,-28.756092,0.975,0.020247,0.000222,7
29643_00373,-28.750441,1.000,0.014567,0.000306,4
29643_00041,-28.718187,1.000,0.010595,0.000258,5


In [120]:
with open("../results/raytune_oja_blocked.pkl", "wb") as f:
    pickle.dump(df, f)

### verify results

In [130]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
dict(sorted(vars(args).items(),key=lambda k: k[0]))

# override defaults 
args.gating = "oja"
args.perform_hebb = False
args.centering = True
args.lrate_hebb = 0.000161
args.lrate_sgd = 0.040964
args.ctx_scaling = 1
# create dataset 
dataset = make_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_model(args, model, optimiser, dataset, logger)

step 0, loss: task a -0.0465, task b -0.0787 | acc: task a 0.5000, task b 0.5500
... n_a: 1 n_b: 7
step 50, loss: task a -4.3483, task b 0.1112 | acc: task a 1.0000, task b 0.5000
... n_a: 1 n_b: 1
step 100, loss: task a -8.1179, task b 0.1366 | acc: task a 1.0000, task b 0.4500
... n_a: 4 n_b: 0
step 150, loss: task a -10.9129, task b 0.0347 | acc: task a 1.0000, task b 0.5000
... n_a: 7 n_b: 0
step 200, loss: task a -12.4492, task b 0.0419 | acc: task a 1.0000, task b 0.5000
... n_a: 4 n_b: 0
step 250, loss: task a -13.2750, task b -0.0152 | acc: task a 1.0000, task b 0.5000
... n_a: 4 n_b: 0
step 300, loss: task a -13.7418, task b 0.0171 | acc: task a 1.0000, task b 0.5000
... n_a: 5 n_b: 0
step 350, loss: task a -14.0349, task b 0.0247 | acc: task a 1.0000, task b 0.5000
... n_a: 4 n_b: 0
step 400, loss: task a -14.2267, task b 0.0083 | acc: task a 1.0000, task b 0.5000
... n_a: 5 n_b: 0
step 450, loss: task a -14.3590, task b 0.0078 | acc: task a 1.0000, task b 0.5000
... n_a: 5 n